In [ ]:
#!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz
!tar xf spark-3.0.0-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install Faker

SSL_INIT


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

import csv
from datetime import datetime, timedelta
from random import random, randint, choice


import numpy
from faker import Faker
from geopy.distance import geodesic as vincenty

from datetime import datetime

import json
import time

from pyspark.sql import SparkSession

In [ ]:
DRIVERS_NUM = 3000
CLIENTS_NUM = 5000
TRIPS_NUM = 400000
BATCH_SIZE = 10*3

TO_DATE = datetime.today()
FROM_DATE = TO_DATE - timedelta(days=30)

MAX_RATE = 5
FEEDBACK_CATEGORY=['politeness' , 'sociability' ,'punctuality']
FEEDBACK_RATE = [-1, 0, 1]
result_file = 'data.txt'
AVG_SPEED_KM_PER_HOUR = 45

hour_probability = {
    0: 0.03,
    1: 0.02,
    2: 0.01,
    3: 0.01,
    4: 0.005,
    5: 0.005,
    6: 0.01,
    7: 0.03,
    8: 0.09,
    9: 0.08,
    10: 0.07,
    11: 0.04,
    12: 0.03,
    13: 0.02,
    14: 0.03,
    15: 0.03,
    16: 0.04,
    17: 0.06,
    18: 0.08,
    19: 0.09,
    20: 0.07,
    21: 0.06,
    22: 0.05,
    23: 0.04,
}
hours = []
probabilities = []
for hour, probability in hour_probability.items():
    hours.append(hour)
    probabilities.append(probability)

commentsFactory = Faker()

In [ ]:
def random_date(start, end):
    """Generate a random datetime between `start` and `end`"""
    random_time =  start + timedelta(
        seconds=randint(0, int((end - start).total_seconds())),
    )
    hour = numpy.random.choice(hours, p=probabilities)
    return random_time.replace(hour=hour)


def flip_a_coin(probability: float) -> bool:
    if probability < 0 or probability > 1:
        raise ValueError("Probability must be between 0 and 1")
    return random.random() < probability

def trip_cost(distance, start_date):
    serving_fee = 30
    price_per_km = 5
    coef = 1
    RUSH_HOURS = {8, 9, 17, 18, 19, 20}
    if start_date.hour in RUSH_HOURS:
        coef = 1.3
    return (serving_fee + distance * price_per_km) * coef


def main(file):
    TRIPS = []

    with open('/content/London postcodes.csv', 'r') as postcodes:
        codes = list(csv.DictReader(postcodes, delimiter=','))
        DESTINATIONS_NUM = len(codes)

    for i in range(1, TRIPS_NUM + 1):
        driver_feedback = []
        client_feedback = []
        client_rate = 0
        driver_rate = 0
        if i % 1000 == 0:
            print(str(i) + ' codes')

        driver = randint(0, DRIVERS_NUM - 1)
        client = randint(0, CLIENTS_NUM - 1)

        start, end = randint(0, DESTINATIONS_NUM - 1), randint(0, DESTINATIONS_NUM - 1)
        if start == end:
            end = (end + i) % DESTINATIONS_NUM

        start_point = (float(codes[start]['Latitude']), float(codes[start]['Longitude']))
        end_point = (float(codes[end]['Latitude']), float(codes[end]['Longitude']))
        distance = float(vincenty(start_point, end_point).kilometers)

        start_date = random_date(FROM_DATE, TO_DATE)
        end_date = start_date + timedelta(hours=distance / AVG_SPEED_KM_PER_HOUR)

        if i % 3 == 0:
            driver_rate = randint(0, 5)
            client_rate = randint(3, 5)

        for j in range(len(FEEDBACK_CATEGORY)):
            feedback = 0
            if i % 4 == 0:
                feedback = choice(FEEDBACK_RATE)
            driver_feedback.insert(j, feedback)

        for j in range(len(FEEDBACK_CATEGORY)):
            feedback = 0
            if i % 4 == 0:
                feedback = choice(FEEDBACK_RATE)
            client_feedback.insert(j, feedback)

        driver_comment = commentsFactory.sentence()

        trip = {
            'driver': driver,
            'client': client,
            'start_point': start_point,
            'end_point': end_point,
            'distance': distance,
            'start_date': start_date.strftime("%Y-%m-%d %H:%M"),
            'end_date': end_date.strftime("%Y-%m-%d %H:%M"),
            'driver_rate': driver_rate,
            'driver_comment': driver_comment,
            'driver_feedback': driver_feedback,
            'client_rate': client_rate,
            'client_feedback': client_feedback,
            'cost': trip_cost(distance, start_date)
        }

        TRIPS.append(trip)

        if i % BATCH_SIZE == 0:
            lines = '\n'.join(str(trip) for trip in TRIPS)
            file.write(lines)
            # append ',\n' after every batch except the last one
            if (i + BATCH_SIZE <= TRIPS_NUM):
                file.write('\n')
            TRIPS = []


if __name__ == "__main__":
    with open(result_file, 'w') as f:
        main(f)



1000 codes
2000 codes
3000 codes
4000 codes
5000 codes
6000 codes
7000 codes
8000 codes
9000 codes
10000 codes
11000 codes
12000 codes
13000 codes
14000 codes
15000 codes
16000 codes
17000 codes
18000 codes
19000 codes
20000 codes
21000 codes
22000 codes
23000 codes
24000 codes
25000 codes
26000 codes
27000 codes
28000 codes
29000 codes
30000 codes
31000 codes
32000 codes
33000 codes
34000 codes
35000 codes
36000 codes
37000 codes
38000 codes
39000 codes
40000 codes
41000 codes
42000 codes
43000 codes
44000 codes
45000 codes
46000 codes
47000 codes
48000 codes
49000 codes
50000 codes
51000 codes
52000 codes
53000 codes
54000 codes
55000 codes
56000 codes
57000 codes
58000 codes
59000 codes
60000 codes
61000 codes
62000 codes
63000 codes
64000 codes
65000 codes
66000 codes
67000 codes
68000 codes
69000 codes
70000 codes
71000 codes
72000 codes
73000 codes
74000 codes
75000 codes
76000 codes
77000 codes
78000 codes
79000 codes
80000 codes
81000 codes
82000 codes
83000 codes
84000 codes
8

In [ ]:
class TaxiDataProcessor:
    
    def __init__(self, df):
        self.df = df
        self.drivers_rating = self.df.filter(lambda trip: trip['driver_rate'] > 0).map(self._trip_driver_rate_map).reduceByKey(self._driver_rate_reduce).map(self._driver_avg_map)
        self.clients_rating = self.df.map(self._trip_client_map).reduceByKey(self._client_reduce).map(self._client_map)
        self.client_feedback = self.df.map(self._rate).reduce(self._client_feedback_reduce)

    @staticmethod
    def _trip_driver_rate_map(trip):
        return trip['driver'], (trip['driver_rate'], 1)

    @staticmethod
    def _driver_rate_reduce(acc, n):
        return acc[0] + n[0], acc[1] + n[1]

    @staticmethod
    def _driver_avg_map(driver):
        avg_rating = round(driver[1][0] / driver[1][1], 2)
        return driver[0], avg_rating

    @staticmethod
    def _trip_client_map(trip):
        return trip['client'], (trip['client_rate'], 1)

    @staticmethod
    def _client_map(client):
        avg_rating = round(client[1][0] / client[1][1], 2)
        return client[0], avg_rating

    @staticmethod
    def _client_reduce(acc, n):
        return acc[0] + n[0], acc[1] + n[1]

    def top_drivers(self, n):
        return self.drivers_rating.takeOrdered(n, key=lambda driver: -driver[1])

    def drivers_rating_lower_than(self, rating):
        return self.drivers_rating.filter(lambda a: a[1] < rating).collect()

    def most_intensive_timeframe(self, n_hours=5):

        return self.df.map(lambda x: (datetime.strptime(x['start_date'], "%Y-%m-%d %H:%M").hour, 1)).\
            reduceByKey(lambda a, b: a + b).\
            takeOrdered(n_hours, key=lambda trip: -trip[1])

    def top_clients(self, n):
        return self.clients_rating.takeOrdered(n, key=lambda client: -client[1])
    
    def count_drivers(self):
        return self.df.map(lambda x: x['driver']).distinct().count()

    def count_clients(self):
        return self.df.map(lambda x: x['client']).distinct().count()

    def driver_comment(self):
        map_result = self.df.map(lambda x: x['driver_comment']).distinct().collect()
        return max(map_result, key=len)


    def top_earners(self, n):
        return self.df.map(lambda x: (x['driver'], x['cost'])) \
            .reduceByKey(lambda x, y: x + y) \
            .takeOrdered(n, lambda x: -x[1])

    @staticmethod
    def _trip_hour_filter(trip):
        night_start = 23
        night_end = 6
        trip_hour = datetime.strptime(trip['start_date'], "%Y-%m-%d %H:%M").hour
        return trip_hour >= night_start or trip_hour <= night_end

    def top_nightwolves(self, n):
        return self.df.filter(self._trip_hour_filter).\
            map(lambda trip: (trip['driver'], 1)).\
            reduceByKey(lambda x, y: x + y).\
            takeOrdered(n, key=lambda driver: -driver[1])

    @staticmethod
    def _trip_driver_map(trip):
        return trip['driver'], (trip['driver_rate'], 1)

    @staticmethod
    def _rate(trip):
        if len(trip['driver_feedback']) != 0:
            return trip['driver_feedback']

    @staticmethod
    def _client_feedback_reduce(acc, n):
        if (acc == 0):
            return [0, 0, 0]
        return [acc[0] + n[0], acc[1] + n[1], acc[2] + n[2]]

    def most_praised_driver_quality(self):
        return FEEDBACK_CATEGORY[self.client_feedback.index(max(self.client_feedback))]
    
    def most_complained_driver_quality(self):
        return FEEDBACK_CATEGORY[self.client_feedback.index(min(self.client_feedback))]

    def most_len_comment(self):
        return self.driver_comment[self.driver_comment.index(max(self.driver_comment, key=len))]

In [ ]:
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option") \
    .getOrCreate()
sc = spark.sparkContext
data_file = "data.txt"
df = sc.textFile(data_file, minPartitions=100).map(lambda x: eval(x))
td = TaxiDataProcessor(df)


top_drivers = td.top_drivers(100)
with open("/content/results/top_drivers.json", "w") as f:
    json.dump(top_drivers, f, indent=4, sort_keys=True)
bad_drivers = td.drivers_rating_lower_than(3.5)
with open("results/bad_drivers.json", "w") as f:
   json.dump(bad_drivers, f, indent=4, sort_keys=True)
n = 0
result_dict = {}
for hour, rides in td.most_intensive_timeframe():
   result_dict[n] = {}
   result_dict[n]['start'] = hour
   result_dict[n]['end'] = hour + 1
   result_dict[n]['qty_rides'] = rides
   n += 1
with open("/content/results/timeframes.json", "w") as f:
   json.dump(result_dict, f, indent=4, sort_keys=True)
top_clients = td.top_clients(50)
with open("/content/results/top_clients.json", "w") as f:
   json.dump(top_clients, f, indent=4, sort_keys=True)
count_drivers = td.count_drivers()
with open("/content/results/count_drivers.json", "w") as f:
   json.dump(count_drivers, f, indent=4, sort_keys=True)
    
count_clients = td.count_clients()
with open("/content/results/count_clients.json", "w") as f:
   json.dump(count_clients, f, indent=4, sort_keys=True)
top_earners = td.top_earners(100)
with open("/content/results/top_earners.json", "w") as f:
   json.dump(top_earners, f, indent=4, sort_keys=True)
top_night_drivers = td.top_nightwolves(50)
with open("/content/results/top_night_drivers.json", "w") as f:
   json.dump(top_night_drivers, f, indent=4, sort_keys=True)
top_praised_drivers = td.most_praised_driver_quality()
with open("/content/results/top_praised_drivers.json", "w") as f:
   json.dump(top_praised_drivers, f, indent=4, sort_keys=True)
top_complained_drivers = td.most_complained_driver_quality()
with open("/content/results/top_complained_drivers.json", "w") as f:
   json.dump(top_complained_drivers, f, indent=4, sort_keys=True)

most_len_comment = td.driver_comment()
with open("/content/results/most_len_comment.json", "w") as f:
    json.dump(most_len_comment, f, indent=4, sort_keys=True)